### Importing data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
ords_prods_merge = pd.read_pickle(r"C:\Users\user\Desktop\Data Analytics\02-12-2023 Instacart Basket Analysis\02 Data\Prepared data\orders_products_combined2.pkl")

### Contents: 
#### 1. Calculating individual department's means
#### 2. Creating loyalty flags
#### 3. Creating flags based on average amount spent
#### 4. Creating flags based on order frequency


###  1. Calculating individual department's means

In [3]:
#Calculating mean of order_number grouped by department_id
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#### By applying this function to entire dataframe we are able to view all of the departments (as not all were represented in our subset) and the means have changed somewhat due to more data being avaliable.

### 2. Creating loyalty flags

In [5]:
#Calculating max orders for each user
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
#Viewing output
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,11.114836,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy,Average orders,10
2,473747,1,3,3,12,21.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy,Most orders,10
3,2254736,1,4,4,7,29.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy,Average orders,10
4,431534,1,5,4,15,28.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy,Most orders,10


In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
#Viewing output
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

#### No relationship between loyalty of customers and prices of products they buy detected

#### New customers breakdown

In [17]:
# Creating a new subset filtering the rows where loyalty_flag is new customer 
df_new = ords_prods_merge[ords_prods_merge['loyalty_flag'].str.contains('New customer')] 

In [18]:
#Viewing output
df_new.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,11.114836,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer
2,473747,1,3,3,12,21.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer
3,2254736,1,4,4,7,29.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy,Average orders,10,New customer
4,431534,1,5,4,15,28.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy,Most orders,10,New customer


In [19]:
#Checking basic price statistics for new customers
df_new[['loyalty_flag', 'prices']].describe()

,prices
count,6.243990e+06
mean,1.329467e+01
std,5.975603e+02
min,1.000000e+00
25%,4.200000e+00
50%,7.400000e+00
75%,1.130000e+01
max,9.999900e+04


#### Regular customers breakdown

In [20]:
# Creating a new subset filtering the rows where loyalty_flag is regular customer 
df_regular = ords_prods_merge[ords_prods_merge['loyalty_flag'].str.contains('Regular customer')] 

In [21]:
#Viewing output
df_regular.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
10,2968173,15,15,1,9,7.0,196,2,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest day,Most orders,22,Regular customer
11,1870022,15,17,2,16,8.0,196,6,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,22,Regular customer
12,1911383,15,18,2,11,7.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,22,Regular customer
13,2715276,15,21,1,9,7.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest day,Most orders,22,Regular customer
14,487368,15,22,1,10,14.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest day,Most orders,22,Regular customer


In [22]:
#Checking basic price statistics for new customers
df_regular[['loyalty_flag', 'prices']].describe()

,prices
count,1.587678e+07
mean,1.249572e+01
std,5.397209e+02
min,1.000000e+00
25%,4.200000e+00
50%,7.400000e+00
75%,1.130000e+01
max,9.999900e+04


#### Loyal customers breakdown

In [24]:
# Creating a new subset filtering the rows where loyalty_flag is loyal customer 
df_loyal = ords_prods_merge[ords_prods_merge['loyalty_flag'].str.contains('Loyal customer')] 

In [25]:
#Viewing output
df_loyal.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
78,2040988,195,1,1,14,11.114836,196,2,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest day,Most orders,58,Loyal customer
79,1680569,195,8,1,9,4.000000,196,2,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest day,Most orders,58,Loyal customer
80,276171,195,19,5,11,3.000000,196,2,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,58,Loyal customer
81,2744976,195,22,5,10,7.000000,196,2,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Most orders,58,Loyal customer
82,2781919,195,25,1,14,4.000000,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest day,Most orders,58,Loyal customer


In [26]:
#Checking basic price statistics for loyal customers
df_loyal[['loyalty_flag', 'prices']].describe()

,prices
count,1.028409e+07
mean,1.038634e+01
std,3.280178e+02
min,1.000000e+00
25%,4.200000e+00
50%,7.400000e+00
75%,1.120000e+01
max,9.999900e+04


### 3. Creating flags based on average amount spent

In [27]:
#Calculating average prices of products individual users by
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.average)

In [28]:
#Viewing output
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price
0,2539329,1,1,2,8,11.114836,196,1,0,both,...,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.000000,196,1,1,both,...,77,7,9.0,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797
2,473747,1,3,3,12,21.000000,196,1,1,both,...,77,7,9.0,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.000000,196,1,1,both,...,77,7,9.0,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797
4,431534,1,5,4,15,28.000000,196,1,1,both,...,77,7,9.0,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797


In [29]:
#Creating a flag to assign spending habits of users
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_habit'] = 'Low spender'

In [30]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_habit'] = 'High spender'

In [31]:
#Viewing output
ords_prods_merge['spending_habit'].value_counts(dropna = False)

spending_habit
Low spender     31770646
High spender      634213
Name: count, dtype: int64

### 4. Creating flags based on order frequency

In [32]:
#Calculating medians od days_since_prior_orders column by individual users
ords_prods_merge['med_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [33]:
#Viewing output
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_habit,med_days
0,2539329,1,1,2,8,11.114836,196,1,0,both,...,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0
1,2398795,1,2,3,7,15.000000,196,1,1,both,...,9.0,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.0
2,473747,1,3,3,12,21.000000,196,1,1,both,...,9.0,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.0
3,2254736,1,4,4,7,29.000000,196,1,1,both,...,9.0,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.0
4,431534,1,5,4,15,28.000000,196,1,1,both,...,9.0,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.0


In [34]:
#Creating a flag to assign frequency to users
ords_prods_merge.loc[ords_prods_merge['med_days'] > 20, 'order_frequency'] = 'Non-frequent customer'

In [35]:
ords_prods_merge.loc[(ords_prods_merge['med_days'] > 10) & (ords_prods_merge['med_days'] <= 20), 'order_frequency'] = 'Regular customer' 

In [36]:
ords_prods_merge.loc[ords_prods_merge['med_days'] <= 10, 'order_frequency'] = 'Frequent customer'

In [37]:
#Viewing output
ords_prods_merge['order_frequency'].value_counts(dropna = False)

order_frequency
Frequent customer        20535136
Regular customer          9168905
Non-frequent customer     2700818
Name: count, dtype: int64

### Exporting

In [38]:
path = r"C:\Users\user\Desktop\Data Analytics\02-12-2023 Instacart Basket Analysis"

In [39]:
#Exporting data to pickle
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined3.pkl'))